In [ ]:
#hide
#skip
! [ -e /content ] && pip install -Uqq mrl-pypi  # upgrade mrl on colab

In [ ]:
# default_exp train.log

# Logging

> Callbacks for logging data

In [ ]:
#hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
# export

from mrl.imports import *
from mrl.core import *
from mrl.torch_imports import *
from mrl.torch_core import *
from mrl.train.callback import *

## Log

The `Log` Callback serves the purpose of logging data generated during a training run. The log holds the following objects of interest:

- `batch_log` - a dictionary of batch-wise logged data. Each key is a string denoting the name of a logged attribute. The values are lists of lists, where each sub-list is the value of the given attribute for each item in a batch. For example `log.batch_log['rewards'][5]` would yield an array of rewards for each item in batch 5

- `timelog` - a dictionary of lists. The keys denote different training steps (`build_buffer`, `sample_batch`, etc) with how long the step took for each batch. To view the times for all training stages, use the `Log.plot_timelog` function.

- `metrics` - a dictionary of lists. Each key is the name of a tracked metric. Each value is a list containing the value of that metric for each batch. Metrics are single scalar values, one value per batch. Metrics can be plotted with the `Log.plot_metrics` function

- `unique_samples` - a set containing every unique sample seen during training. This can be used to quickly reference if a sample has been seen before

- `df` - a dataframe containing every unique sample and every `batch_log` value associated with that sample

In [ ]:
# export

class Log(Callback):
    def __init__(self):
        super().__init__(name='log', order=100)
        
        self.pbar = None
        self.iterations = 0
        self.metrics = {}
        
        self.batch_log = {}
        self.timelog = defaultdict(list)
        
        self.report = 1
        self.unique_samples = {}
                
        self.add_metric('rewards')
        self.add_metric('rewards_final')
        self.add_metric('new')
        self.add_metric('diversity')
        self.add_metric('bs')
        
        self.add_log('samples')
        self.add_log('sources')
        self.add_log('rewards')
        self.add_log('rewards_final')
        
    def setup(self):
        self.df = pd.DataFrame(self.batch_log)
        
    def before_train(self):
        cols = ['iterations'] + list(self.metrics.keys())
        if self.pbar is None:
            print('\t'.join(cols))
        else:
            self.pbar.write(cols, table=True)
            
    def add_metric(self, name):
        if not name in self.metrics.keys():
            self.metrics[name] = []
        
    def add_log(self, name):
        if not name in self.batch_log.keys():
            self.batch_log[name] = []
            
    def update_metric(self, name, value):
        self.metrics[name].append(value)

    def after_sample(self):
        env = self.environment
        batch_state = env.batch_state
        samples = batch_state.samples
        batch_state.rewards = to_device(torch.zeros(len(samples)))
        
        new = np.array([not i in self.unique_samples for i in samples])
        
        self.update_metric('new', new.mean())
            
        if len(samples)>0:
            diversity = len(set(samples))/len(samples)
        else:
            diversity = 0.
        self.environment.log.update_metric('diversity', diversity)
        
        self.environment.log.update_metric('bs', len(batch_state.samples))
            
    def after_compute_reward(self):
        env = self.environment
        batch_state = env.batch_state
        samples = batch_state.samples
        rewards = batch_state.rewards
        batch_state.rewards_final = rewards.clone().detach()
        
        rewards = rewards.detach().cpu().numpy()
        
        self.update_metric('rewards', rewards.mean())
        
        for i in range(len(samples)):
            if not samples[i] in self.unique_samples:
                self.unique_samples[samples[i]] = rewards[i]
                
    def after_reward_modification(self):
        env = self.environment
        rewards = env.batch_state.rewards_final.detach().cpu().numpy()
        self.update_metric('rewards_final', rewards.mean())
        
                
    def update_log(self):
        env = self.environment
        batch_state = env.batch_state
        samples = batch_state.samples
        update_dict = {}

        for key in self.batch_log.keys():
            items = batch_state[key]
            if isinstance(items, torch.Tensor):
                items = items.detach().cpu().numpy()
            self.batch_log[key].append(items)
            update_dict[key] = items
            
        new_df = pd.DataFrame(update_dict)
        repeats = new_df.samples.isin(self.df.samples)
        new_df = new_df[~repeats]
            
        self.df = self.df.append(new_df, ignore_index=True)
        
        if self.iterations%10==0 and self.iterations>0:
            self.df.drop_duplicates(subset='samples', inplace=True)
            
    def report_batch(self):
        outputs = [f'{self.iterations}']
        if self.iterations%self.report==0:
            
            for k,v in self.metrics.items():
                val = v[-1]

                if type(val)==int:
                    val = f'{val}'
                elif type(val)==str:
                    val = val
                else:
                    val = f'{val:.3f}'

                outputs.append(val)

            if self.pbar is None:
                print('\t'.join(outputs))
            else:
                self.pbar.write(outputs, table=True)
            
        self.iterations += 1
        
    def after_batch(self):
        self.update_log()
        self.report_batch()
        
    def get_df(self):
        return log_to_df(self.batch_log)
    
    def plot_metrics(self, cols=4, smooth=True):
        self.plot_dict(self.metrics, cols=cols, smooth=smooth)
            
    def plot_timelog(self, cols=4, smooth=True):
        self.plot_dict(self.timelog, cols=cols, smooth=smooth)


In [ ]:
# export

def log_to_df(log, keys=None):
    batch = 0
    output_dict = defaultdict(list)
    
    if keys is None:
        keys = list(log.keys())
    
    items = log[keys[0]]
    for item in items:
        output_dict['batch'] += [batch]*len(item)
        batch += 1
        
    for key in keys:
        output_dict[key] = flatten_list_of_lists(log[key])

    return pd.DataFrame(output_dict)

## Custom Metrics Logging

Adding new items to metric tracking and batch logging is easy.

Use the `add_log` to add a new term to the batch log. At some point during the batch, add the values to be logged to the current `BatchState` with an attribute name that matches the name added to the log. The batch log will automatically add the values to the batch log.

Use `add_metric` to add a new term to the metric log. At some point during the batch, compute the metric you wish to log. Then use `Log.update_metric` to add the value to the metric log.

Here's an outline implementation:

```
class MyCallback(Callback):
    def __init__(self):
        super().__init__(name='my_callback')
        
    def setup(self):
        log = self.environment.log
        log.add_log(self.name) # adding new term to batch log
        log.add_metric(self.name) # adding new term to metrics
        
    def compute_reward(self):
        # make tensor of dummy rewards
        batch_state = self.environment.batch_state
        bs = len(batch_state.samples)
        rewards = to_device(torch.ones(bs).float())
        
        batch_state.rewards += rewards
        batch_state[self.name] = rewards # this is the value the batch log will pick up
        
    def after_compute_reward(self):
        log = self.environment.log
        batch_state = self.environment.batch_state
        
        my_callback_rewards = batch_state[self.name]
        my_metric = my_callback_rewards.mean()
        
        log.update_metric(self.name, my_metric.detach().cpu().numpy()) # update metric value

```

## Log Callbacks

Several logging related callbacks

In [ ]:
# export

class StatsCallback(Callback):
    '''
    StatsCallback - base class for callbacks related to calculating 
    stats from batches
    
    Inputs:
    
    - `batch_attribute str`: attribute to grab from the log
    
    - `grabname Optional[str]`: if passed, the `batch_attribute` values 
    will be subset for those where `source==grabname`
    
    - `include_buffer bool`: if True, values sourced from the buffer 
    that match `grabname` will be included
    
    - `name str`: callback name
    
    - `order int`: callback order
    '''
    def __init__(self, batch_attribute, grabname=None, include_buffer=True, 
                     name='stats', order=20):
        super().__init__(name=name, order=order)
        
        self.grabname = grabname
        self.batch_attribute = batch_attribute
        self.include_buffer = include_buffer

    def get_values(self):
        batch_state = self.environment.batch_state
        sources = np.array(batch_state.sources)
        
        if self.include_buffer:
            sources = np.array([i.replace('_buffer', '') for i in sources])
        
        values = batch_state[self.batch_attribute]
        
        if self.grabname is not None:
            source_mask = sources==self.grabname
            if source_mask.sum()>0:
                values = values[source_mask]
            else:
                values = np.array([0.])
            
        if isinstance(values, torch.Tensor):
            values = values.detach().cpu().numpy()
            
        return values

In [ ]:
# export

class MaxCallback(StatsCallback):
    '''
    MaxCallback - adds a metric tracking the maximum of 
    `batch_attribute`, subset by `grabname`, printed every 
    batch report
    
    Inputs:
    
    - `batch_attribute str`: attribute to grab from the log
    
    - `grabname Optional[str]`: if passed, the `batch_attribute` values 
    will be subset for those where `source==grabname`
    
    - `include_buffer bool`: if True, values sourced from the buffer 
    that match `grabname` will be included
    '''
    def __init__(self, batch_attribute, grabname, include_buffer=True):
        
        if grabname is None:
            name = f'{batch_attribute}_max'
        else:
            name = f'{batch_attribute}_{grabname}_max'
        
        super().__init__(batch_attribute, grabname, 
                         include_buffer=include_buffer, name=name)
        
        
    def setup(self):
        log = self.environment.log
        log.add_metric(self.name)
        
    def after_compute_reward(self):
        
        values = self.get_values()
        self.environment.log.update_metric(self.name, values.max())

In [ ]:
# export

class MinCallback(StatsCallback):
    '''
    MinCallback - adds a metric tracking the minimum of 
    `batch_attribute`, subset by `grabname`, printed every 
    batch report
    
    Inputs:
    
    - `batch_attribute str`: attribute to grab from the log
    
    - `grabname Optional[str]`: if passed, the `batch_attribute` values 
    will be subset for those where `source==grabname`
    
    - `include_buffer bool`: if True, values sourced from the buffer 
    that match `grabname` will be included
    '''
    def __init__(self, batch_attribute, grabname, include_buffer=True):
        
        if grabname is None:
            name = f'{batch_attribute}_min'
        else:
            name = f'{batch_attribute}_{grabname}_min'
        
        super().__init__(batch_attribute, grabname, 
                         include_buffer=include_buffer, name=name)
        
        
    def setup(self):
        log = self.environment.log
        log.add_metric(self.name)
        
    def after_compute_reward(self):
        
        values = self.get_values()
        self.environment.log.update_metric(self.name, values.min())

In [ ]:
# export

class MeanCallback(StatsCallback):
    '''
    MeanCallback - adds a metric tracking the mean of 
    `batch_attribute`, subset by `grabname`, printed every 
    batch report
    
    Inputs:
    
    - `batch_attribute str`: attribute to grab from the log
    
    - `grabname Optional[str]`: if passed, the `batch_attribute` values 
    will be subset for those where `source==grabname`
    
    - `include_buffer bool`: if True, values sourced from the buffer 
    that match `grabname` will be included
    '''
    def __init__(self, batch_attribute, grabname, include_buffer=True):
        
        if grabname is None:
            name = f'{batch_attribute}_max'
        else:
            name = f'{batch_attribute}_{grabname}_max'
        
        super().__init__(batch_attribute, grabname, 
                         include_buffer=include_buffer, name=name)
        
        
    def setup(self):
        log = self.environment.log
        log.add_metric(self.name)
        
    def after_compute_reward(self):
        
        values = self.get_values()
        self.environment.log.update_metric(self.name, values.mean())

In [ ]:
# export

class PercentileCallback(StatsCallback):
    '''
    PercentileCallback - adds a metric tracking the `percentile` 
    percentile value of `batch_attribute`, subset by `grabname`, 
    printed every batch report
    
    Inputs:
    
    - `batch_attribute str`: attribute to grab from the log
    
    - `grabname Optional[str]`: if passed, the `batch_attribute` values 
    will be subset for those where `source==grabname`
    
    - `percentile str`: what percentile value to use
    
    - `include_buffer bool`: if True, values sourced from the buffer 
    that match `grabname` will be included
    '''
    def __init__(self, batch_attribute, grabname, percentile, include_buffer=True):
        
        if grabname is None:
            name = f'{batch_attribute}_p{percentile}'
        else:
            name = f'{batch_attribute}_{grabname}_p{percentile}'
        
        super().__init__(batch_attribute, grabname, 
                         include_buffer=include_buffer, name=name)
        self.percentile = percentile
        
    def setup(self):
        log = self.environment.log
        log.add_metric(self.name)
        
    def after_compute_reward(self):
        
        values = self.get_values()
        self.environment.log.update_metric(self.name, np.percentile(values, self.percentile))
        

In [ ]:
# export

class SaveLogDF(Callback):
    '''
    SaveLogDF - periodically saves the Log
    dataframe during training
    
    Inputs:
    
    - `frequency int`: how often to save
    
    - `save_path str`: directory to save 
    files to
    '''
    def __init__(self, frequency, save_path):
        super().__init__(name='save_log')
        self.frequency = frequency
        self.save_path = save_path
        
    def after_batch(self):
        log = self.environment.log
        if (log.iterations%self.frequency)==0 and log.iterations>0:
            log.df.to_csv(f'{self.save_path}/log_df_{log.iterations}.csv', index=False)
        

In [ ]:
# hide
from nbdev.export import notebook2script; notebook2script()